In [1]:
### Analyze MOFA MODEL Results
### Associate the generated factors with sample meta-data covariates and plot the top features per factor

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/jove_r//lib/R/library"


‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.4; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::betwee

In [3]:
source('MS2_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


In [4]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [5]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [6]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [7]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [8]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [9]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [10]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [11]:
## Downstream Analysis Configurations 

In [12]:
factor_configs = read.csv( 'configurations/04_Factor_Analysis_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/04_Factor_Analysis_Configs.csv'”


In [13]:
head(factor_configs,2)

,configuration_name,mofa_result_name,relevant_factors,numeric_covariates,categorical_covariates,top_variable_thres.
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,config1,config1,"Factor1,Factor2,Factor3,Factor4,Factor5,Factor6","CK,CRP","measurement,classification,tp_outcome",0.005


In [14]:
## Get Type color codes from previous script

In [15]:
type_color_codes = read.csv( 'configurations/03_Type_Color_Codes.csv', sep = ',')

In [16]:
head(type_color_codes,2)

,X,color_code
,<int>,<chr>
1,1,#FF7F50
2,2,#D95F02


In [17]:
type_color_codes$color_code

[1] "#FF7F50" "#D95F02" "#377EB8" "#E6AB02" "#31A354" "#7570B3" "#E7298A"
 [8] "#66A61E" "#A6761D" "#666666" "#E41A1C" "#4DAF4A" "#984EA3" "#FF7F00"
[15] "#FFFF33" "#A65628" "#F781BF"

# Load Data 

## MOFA Input

In [ ]:
data_list = list()

In [ ]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    data_name = factor_configs$configuration_name[i]
    
    # Load data
    path = paste0(result_path, '/02_results/02_Combined_Data_',data_name,'_INTEGRATED.csv')
    data_long = read.csv(path)
    data_long$X = NULL
    print(file.info(path)$mtime)
    print(path)
    
    # save to list
    data_list[[i]] = data_long
    }

In [ ]:
## Example of loaded data

In [ ]:
head(data_long,2)

## MOFA Model

### Load estimated model

In [ ]:
model_list = list()

In [ ]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
 
    # Load model
    model_name =  paste0("03_MOFA_MODEL_",mofa_name, '.hdf5')
    outfile = file.path( paste0(result_path, '/03_results/',  model_name) )
    model = load_model(outfile, verbose = TRUE)
    
    print(file.info(outfile)$mtime)
    print(outfile)
    
    # save to list
    model_list[[i]] = model
    }
    

### Factor Data

In [ ]:
factor_data_list = list()

In [ ]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
    
    # load data
    path = paste0(result_path, '/03_results/', '03_Factor_Data_', mofa_name, '.csv')
    factors = read.csv(path)
    print(file.info(path)$mtime)
    print(path)
    
    # Save to list
    factor_data_list[[i]] = factors
    }
    

In [ ]:
head(factors,2)

### Weight Data

In [ ]:
weight_data_list = list()

In [ ]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
    
    # laod data
    path = paste0(result_path, '/03_results/', '03_Weight_Data_',  mofa_name, '.csv')
    weight_data = read.csv(path)
    
    print(file.info(path)$mtime)
    print(path)
    
    # save to list
    weight_data_list[[i]] = weight_data
    }
    
    

## Sample Meta Data

In [29]:
## Sample meta-data that contains covariates that should be associated to factor data

In [30]:
sample_data = read.csv(paste0(data_path, 'Prepared_Sample_Meta_Data', '.csv'))
sample_data$X = NULL

In [31]:
head(sample_data,2)

,sample_id,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,Control,healthy,NA,NA,43,0.4,NA,NA
2,k10,Control,coronary_vessel_disease,NA,NA,NA,0.2,NA,3648632


# Downstream Analysis of generated model

# Extract and prepare data for plots

## Merge factors and sample data

In [32]:
factor_data_processed = list()

In [33]:
for( i in 1:length(factor_data_list)){
    
    ## Transform data
    factors_long = melt(factor_data_list[[i]], id.vars = 'sample_id')
    
    ## Add sample data info
    merged_data_long = merge(factors_long, sample_data, by.x = 'sample_id', by.y = 'sample_id')
    
     ### Filter on relevant factors
     relevant_factors = unlist(str_split(factor_configs$relevant_factors[i], ','))  # get relevant factor from configuration data
     merged_data_long = merged_data_long[merged_data_long$variable %in% relevant_factors,]
    
     factor_data_processed[[i]] = merged_data_long
    }

In [34]:
head(factor_data_processed[[i]],2)

,sample_id,variable,value,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,Factor1,0.6644081,Control,healthy,NA,NA,43,0.4,NA,NA
2,k1,Factor3,1.1575432,Control,healthy,NA,NA,43,0.4,NA,NA


## Extract explained variance for plotting

In [35]:
explained_variance = lapply(model_list, function(x){
    
    # extract variance per factor from model
    data = x@cache$variance_explained$r2_per_factor[[1]]
    
    # extract total variance from model
    total_variance = data.frame( view = rownames(x@cache[["variance_explained"]]$r2_total$group1,2),
                             total_variance = x@cache[["variance_explained"]]$r2_total$group1)
    
    # extract total variance per factor
    total_variance_factor = data.frame(factor = names(rowMeans(x@cache$variance_explained$r2_per_factor[[1]])),
                                   mean_variance = rowMeans(x@cache$variance_explained$r2_per_factor[[1]]))
    
    data = melt(data)
    # merge different variance values
    data = merge(data, total_variance, by.x = 'Var2', by.y = 'view')
    
    })

In [36]:
head(explained_variance[[1]],2)

,Var2,Var1,value,total_variance
,<fct>,<fct>,<dbl>,<dbl>
1,B.cell,Factor1,6.092751,36.95212
2,B.cell,Factor2,4.057193,36.95212


## Prepare weight data

In [37]:
feature_weights_list = lapply(weight_data_list, function(x){
    feature_weights_long = melt(x, id.vars = c('variable_name', 'type'))
    
    # adjust formatting of columns
    feature_weights_long$view = feature_weights_long$type
    feature_weights_long$gene = str_replace(feature_weights_long$variable_name, '.*__', '')
    
    return(feature_weights_long)
    })
    

In [38]:
head(feature_weights_list[[1]],2)

,variable_name,type,variable,value,view,gene
,<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>
1,B.cell__ACTB,B.cell,Factor1,-0.21732116,B.cell,ACTB
2,B.cell__ACTG1,B.cell,Factor1,-0.03822634,B.cell,ACTG1


## Get top features per factor and amounts for diff thresholds

In [39]:
geneset_oi_amounts_list = list()

In [40]:
for(i in 1:length(feature_weights_list)){
    
    feature_weights_long = feature_weights_list[[i]]
    
    # get threshold for top variables from configuration
    top_variable_perc = factor_configs$top_variable_thres[i]
    
    
    ## Define amont of top genes per fraction 
    geneset_oi_pos_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange( desc(value),  .by_group = TRUE)  %>% top_frac(  as.numeric(top_variable_perc), value)
    geneset_oi_pos_per_factor_analyze$direction = 'positive'
    
    geneset_oi_neg_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange(desc(value),  .by_group = TRUE)  %>% top_frac( - as.numeric(top_variable_perc), value)
    geneset_oi_neg_per_factor_analyze$direction = 'negative'
    
    geneset_oi_analyze = rbind(geneset_oi_pos_per_factor_analyze, geneset_oi_neg_per_factor_analyze)
    geneset_oi_analyze$fraction =  as.numeric(top_variable_perc)

    
    ## Calculate the amount of top features per type
    dimensions = unique(feature_weights_long[,c('view', 'variable')])
    
    amount_geneset_oi_type = geneset_oi_analyze %>% group_by(type, view, variable) %>% dplyr::count()
    amount_geneset_oi_type = merge(dimensions, amount_geneset_oi_type, all.x = TRUE) # to avoid missing dimensions
    amount_geneset_oi_type$fraction = as.numeric(top_variable_perc)
    
    geneset_oi_amounts = amount_geneset_oi_type
    
    ## Calculate the total amount of features per type
    features_per_type = feature_weights_long %>% group_by(type, view, variable) %>% dplyr::count()
    colnames(features_per_type) = c('type', 'view', 'variable', 'amount_features')
    
    ## Merge and calculate percentage
    geneset_oi_amounts = merge(  geneset_oi_amounts,features_per_type, all.x = TRUE)
    geneset_oi_amounts$percentage = geneset_oi_amounts$n / geneset_oi_amounts$amount_features
    
    ## Adjust NA's
    geneset_oi_amounts[is.na(geneset_oi_amounts)] = 0 # NA when there are no features for this dimension among top %
    
    ## save for the threshold/ config
    geneset_oi_amounts_list[[i]] = geneset_oi_amounts
    }
    
    
    
    

In [41]:
head(    geneset_oi_amounts_list[[i]],2)

,view,variable,type,n,fraction,amount_features,percentage
,<chr>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,B.cell,Factor1,0,0,0.005,0,0.000000000
2,B.cell,Factor2,B.cell,1,0.005,686,0.001457726


# Plots

## Investigate relationship of factors with numeric values

In [42]:
head(factor_data_processed[[1]],2)

,sample_id,variable,value,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,Factor1,0.6644081,Control,healthy,NA,NA,43,0.4,NA,NA
2,k1,Factor3,1.1575432,Control,healthy,NA,NA,43,0.4,NA,NA


In [43]:
figure_name = "FIG04_Factor_Association_with_numeric_features_"

In [44]:
# Sizes of the plot
width_par = 8.07
height_par = 3.5

In [45]:
for(j in 1:length(factor_data_processed)){
    ## get the variables
    numeric_variables = unlist(str_split(factor_configs$numeric_covariates[j], ','))
    
    
    ## calculate correlations
    cor_plot = list()
    
    ## get the data
    merged_data_long = factor_data_processed[[j]]
    
    ### calculate correlations
    for(i in numeric_variables){
        cor_plot[[i]] = ggplot(merged_data_long, aes(x = value, y = get(i))) + facet_wrap(.~ variable, scale = 'free') +
        geom_point(size = 0.2) + plot_config + geom_smooth(method='lm', col = 'blue3', se = FALSE) + stat_cor(method = 'pearson') + ylab(i)


        }
    
    
    ### Plot the scatterplots
    
    # get the name for saving
    mofa_name = factor_configs$mofa_result_name[j]
    
    pdf(paste0('figures/04_figures/', figure_name,mofa_name, '.pdf'), width =width_par, height =height_par, onefile = TRUE)
    for (i in names(cor_plot)) {
      print(cor_plot[[i]])  
    }
    dev.off()
    }
    
    

`geom_smooth()` using formula = 'y ~ x'
Warning message:
“Removed 48 rows containing non-finite values (`stat_smooth()`).”
Warning message:
“Removed 48 rows containing non-finite values (`stat_cor()`).”
Warning message:
“Removed 48 rows containing missing values (`geom_point()`).”
`geom_smooth()` using formula = 'y ~ x'
Warning message:
“Removed 150 rows containing non-finite values (`stat_smooth()`).”
Warning message:
“Removed 150 rows containing non-finite values (`stat_cor()`).”
Warning message:
“Removed 150 rows containing missing values (`geom_point()`).”


## Investigate relationship with categorical values

In [46]:
# Specific Text Descriptions:
xlabel = xlab('') 
ylabel = ylab('Factor Value')

In [47]:
# Specify Figure Name
figure_name = paste0("FIG04_Factor_Association_with_categorical_features_")

In [48]:
# Sizes of the plot
width_par = 8.07
height_par = 3

In [49]:
for(j in 1:length(factor_data_processed)){
    ## get the variables
    categorical_variables = unlist(str_split(factor_configs$categorical_covariates[j], ','))
    
    
    ## calculate correlations
    cor_plot = list()
    
    ## get the data
    merged_data_long = factor_data_processed[[j]]
    
    
    ## Plot as PDF
    mofa_name = factor_configs$mofa_result_name[j]
    
    pdf(paste0('figures/04_figures/', figure_name, mofa_name, '.pdf'), width =width_par, height =height_par)
    for(i in categorical_variables){
        variable = i
        merged_data_long$condition = merged_data_long[,variable]
        vis_data = merged_data_long
        g = ggplot(vis_data[!is.na(vis_data$condition),], aes(x=condition, y=value, col = condition)) + facet_grid(.~variable) +
            plot_config +
            xlabel + 
            ylabel + #colors_list[[i]]   +
            ggtitle('Pattern of factor values') + 
            theme(legend.position = "bottom", axis.text.x = element_blank())+
            geom_boxplot(outlier.size = 0.05)   + geom_point(size = 0.5) + if(i %in% names(colors_list)){colors_list[[i]]} else{colors_list[['default']]}

        print(g)

        }

    dev.off()
    
    }

## Feature Overview per Factor

In [50]:
## generate overview of amount of top features

In [51]:
## Heatmap with explained variance

In [52]:
variance_plots = list()

In [53]:
for(j in 1:nrow(factor_configs)){

    ## get the relevant factor and top variable fraction
    factor_var = unlist(str_split(factor_configs$relevant_factors[j], ','))
    
    
    ## Explained Variance Heatmap Plot (for each factor)
    explained_variance_heatmap = list()
    for(i in factor_var ){
        data = explained_variance[[j]]   # get prepared variance data
        data$Var2 = as.character(data$Var2)
        data$Var2 = factor(data$Var2, levels = sort(unique(data$Var2)))  # recode to ensure right ordering
        
        data_plot = data[data$Var1 == i,]
        data_plot$Var1 = 'Explained'

        explained_variance_heatmap[[i]] = ggplot() + scale_fill_gradient(low="white", high="black") + 
        ylabel + xlabel + plot_config_heatmap +  theme(axis.text.y = element_text(hjust = 0, vjust = 0.5)) +
        geom_tile(data = data_plot, mapping = aes(Var1,  Var2, fill= value)
                 )  +
        ggtitle(i)
        }
    
    variance_plots[[j]] =  explained_variance_heatmap
    }

In [54]:
#variance_plots[[1]]

In [55]:
## Barplots with top features

In [56]:
barplot_top_features_percentages = list()
barplot_top_features_absolute = list()

In [57]:
for(j in 1:nrow(factor_configs)){
    
    # get configurations
    top_var_fraction = factor_configs$top_variable_thres[j]
    factor_var = unlist(str_split(factor_configs$relevant_factors[j], ','))
    
    ## Barplots with top features per factor
    
    ## 1: Percentages
    xlabel = xlab('View') 
    ylabel = ylab('Percentage of total features')
    
    percentage_plot_1_perc = list()
    
    for(i in factor_var ){

            geneset_oi_amounts = geneset_oi_amounts_list[[j]]
            geneset_oi_amounts$view  = as.character(geneset_oi_amounts$view)
            geneset_oi_amounts$view = factor(geneset_oi_amounts$view, levels = sort(unique(geneset_oi_amounts$view))) # recode to ensure right ordering

            percentage_plot_1_perc[[i]] = ggplot(data = geneset_oi_amounts[(geneset_oi_amounts$variable == i),], aes(x = view, y = percentage*100, fill = view)) +
            xlabel + 
            ylabel + 
            plot_config + 
            geom_bar(stat="identity") + coord_flip() + theme(legend.position = 'none') +
            ggtitle(paste0('Top ', 2*as.numeric( top_var_fraction) *100, '% of features')) +
            geom_hline(yintercept = 2*as.numeric( top_var_fraction)*100, 
                    color = "black", size=1) + scale_fill_manual(values = type_color_codes$color_code)
        
    }
    barplot_top_features_percentages[[j]] =  percentage_plot_1_perc 
    
    
    
    ## 2: Absolute Values
    
    xlabel = xlab('View') 
    ylabel = ylab('Amount features')
    
    absolute_plot_1_perc = list()
    
    # one selected threshold + absolute amount
    for(i in factor_var ){
            geneset_oi_amounts = geneset_oi_amounts_list[[j]]
            geneset_oi_amounts$view = as.character(geneset_oi_amounts$view)
            geneset_oi_amounts$view = factor(geneset_oi_amounts$view, levels = sort(unique(geneset_oi_amounts$view)))# recode to ensure right ordering

            absolute_plot_1_perc[[i]] = ggplot(data = geneset_oi_amounts[(geneset_oi_amounts$variable == i),], aes(x = view, y = n, fill = view)) +
            xlabel + 
            ylabel + 
            plot_config + 
            geom_bar(stat="identity") + coord_flip()  + theme(legend.position = 'none')+ 
            ggtitle(paste0('Top ', 2*as.numeric( top_var_fraction) *100, '% of features')) + scale_fill_manual(values = type_color_codes$color_code) # TBD: maybe improve even with default value + specifying all colors via table
        }
    
    barplot_top_features_absolute[[j]] = absolute_plot_1_perc
       

}  

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


In [58]:
#barplot_top_features_percentages[[1]]

In [59]:
## Combine the plots

In [60]:
figure_name = paste0( "FIG04_Top_Feature_Overview_per_Factor")

In [61]:
# Sizes of the plot
width_par = 8.07
height_par =2.8

In [62]:
for(j in 1:nrow(factor_configs)){
    mofa_name = factor_configs$mofa_result_name[j]
    
    # get the relvant plot
    explained_variance_heatmap = variance_plots[[j]]
    absolute_plot_1_perc = barplot_top_features_absolute[[j]]
    percentage_plot_1_perc = barplot_top_features_percentages[[j]]


    pdf(paste0('figures/04_figures/', figure_name, '_',   mofa_name, '.pdf'), width =width_par, height =height_par)
    for( i in 1:length(explained_variance_heatmap)){
        legend = get_legend(explained_variance_heatmap[[i]])

        combined1 = ggarrange(explained_variance_heatmap[[i]] + theme(legend.position = 'none'),
                             absolute_plot_1_perc[[i]] + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank() ), 
                             percentage_plot_1_perc[[i]] + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank() ),  
                             nrow=1, widths = c(2.2,1,1))
        combined1 = annotate_figure(combined1, right = legend)

        print( combined1)
        }
    dev.off()   
    }